In [ ]:
import numpy as np
from spade.behaviour import CyclicBehaviour

class Concensus_action(CyclicBehaviour):
    def __init__(self, agent):
        super().__init__(agent)

        # Initial weights for the agents
        # The weights are the influence of each agent on the consensus
        self.weights = np.array([
            [0.7,0.3],
            [0.4,0.6],
        ])

    async def run(self):
        msg = await self.receive(timeout=10)
        if msg:
            body = msg.body
            if body.startswith("harmful"):
                content = body[8:-1]
                msg_id, text, textScore = content.split(",")
                textScore = float(textScore)
                await self.agent.bdi.set_belief("harmful", msg_id, text, textScore)

            elif body.startswith("behavioral"):
                content = body[11:-1]
                msg_id, user, userScore = content.split(",")
                userScore = float(userScore)
                await self.agent.bdi.set_belief("behavioral", msg_id, user, userScore)

    @agent3.bdi.on_event("!python_consensus")
    async def calculate_consensus(self, msg_id, text_score, user_score):
        opinions = np.array([float(text_score), float(user_score)])
        consensus = french_degroot(self.weights, opinions)
        if consensus > 0.5:
            action = "keep"
        elif consensus > 0:
            action = "remove"
        else:
            action = "ban"
        await self.agent.bdi.set_belief("final_decision", msg_id, action, consensus)
        return True

    async def consensus(self):
        for msg_id, data in list(self.scores.items()):
            if "text_score" in data and "user_score" in data:
                textScore = data["text_score"]
                userScore = data["user_score"]
                # Calculate the consensus score
                consensus = french_degroot(self.weights, [textScore, userScore])
                action = self.decide_action(consensus)
                self.agent.bdi.set_belief("final_decision", msg_id, action, consensus)
                #await self.take_action(msg_id, action)
                # Set belief about the consensus score
                del self.scores[msg_id]
    

def train(weights, text_score, user_score, actual_result):
    # This function is used to train the weights based on the actual result
    
    opinions = np.array([float(text_score), float(user_score)])
    results = check_correctness(opinions, actual_result)
    adjust_weights(weights, results)

def french_degroot(weights, opinions):
    threshold = 0.01
    # protect the original opinions (optional)
    # opinions = opinions.copy()  

    for _ in range(100):
        new_opinions = np.dot(weights, opinions)
        if np.all(np.abs(new_opinions - opinions) < threshold):
            break
        opinions = new_opinions
    
    return opinions[0]

def check_correctness(opinions, actual_result):
    # This function checks if the agents' opinions match the actual result

    classification = []
    for i in range(len(opinions)):
        if opinions[i] > 0.5:
            classification.append("keep")
        elif opinions[i] > 0:
            classification.append("remove")
        else:
            classification.append("ban")

    results = [classification[i] == actual_result for i in range(len(classification))]
    return results

def adjust_weights(weights, results):
    # Adjust an agents influence based on results

    # If the results are the same, no change is needed
    if results[0] == results[1]:
        pass
    
    # If the results are different, adjust the weights
    # n = learning rate
    n = 0.1
    for i in range(len(weights)):
        if results[i]:
            weights[i] += n * (1 - weights[i])
        else:
            weights[i] -= n * weights[i]
    # Normalize the weights
    weights = weights / np.sum(weights)

def decide_action(msg_score, user_score):
    x = np.array([msg_score, user_score])
    consensus = french_degroot(x)
    if consensus > 0.5:
        return "keep"
    elif consensus > 0:
        return "remove"
    else:
        return "ban"

In [6]:
import numpy as np

def french_degroot(weights, opinions):
    threshold = 0.01

    for _ in range(100):
        new_opinions = np.dot(weights, opinions)
        if np.all(np.abs(new_opinions - opinions) < threshold):
            break
        opinions = new_opinions
    
    return opinions[0]

weights = np.array([
    [0.9, 0.1],
    [0.4, 0.6],
])
opinions = np.array([0.8, 0.2])

final = french_degroot(weights, opinions)
print("Final consensus score:", final)

Final consensus score: 0.6837500000000004
